### LSTM 재학습
- 과적합 문제를 해결하기 위해 train data를 늘려 가장 정확도가 높았던 LSTM 모델을 재학습
- Train:Test 비율을 9:1 로 수정

In [1]:
# 시간 측정 
!pip install ipython-autotime
%load_ext autotime

time: 0 ns (started: 2021-12-19 20:05:43 +09:00)


In [31]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
from plotnine import *
import re
import torch
import tqdm
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split 
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import os, random
import tensorflow as tf

time: 0 ns (started: 2021-12-19 20:10:59 +09:00)


### 스팀 리뷰 데이터

In [3]:
steam_data = pd.read_csv('steam.txt',sep='\t', names=['label', 'text'])

time: 266 ms (started: 2021-12-19 20:05:55 +09:00)


In [4]:
# 부정은 0, 긍정은 1로 표시
steam_data

,label,text
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요
...,...,...
99995,0,한글화해주면 10개산다
99996,0,개쌉노잼 ㅋㅋ
99997,0,노잼이네요... 30분하고 지웠어요...
99998,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기. 영어는 그리 어렵지 않습니다.


time: 16 ms (started: 2021-12-19 20:05:55 +09:00)


In [5]:
#한글이 아니면 빈 문자열로 바꾸기
steam_data['text'] = steam_data['text'].str.replace('[^가-힣\s]','',regex=True)

time: 218 ms (started: 2021-12-19 20:05:56 +09:00)


In [6]:
steam_data

,label,text
0,0,노래가 너무 적음
1,0,돌겠네 진짜 황숙아 어크 공장 그만 돌려라 죽는다
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악차악차악 정말 이래서 왕국을 되찾을 수 있는거야
4,1,시간 때우기에 좋음 도전과제는 시간이면 다 깰 수 있어요
...,...,...
99995,0,한글화해주면 개산다
99996,0,개쌉노잼
99997,0,노잼이네요 분하고 지웠어요
99998,1,야생을 사랑하는 사람들을 위한 짧지만 여운이 남는 이야기 영어는 그리 어렵지 않습니다


time: 16 ms (started: 2021-12-19 20:05:56 +09:00)


In [7]:
train_data, test_data = train_test_split(steam_data, test_size=0.1, random_state=0)

time: 31 ms (started: 2021-12-19 20:05:56 +09:00)


In [8]:
print("train set: {}개".format(len(train_data)))
print("test set: {}개".format(len(test_data)))

train set: 90000개
test set: 10000개
time: 0 ns (started: 2021-12-19 20:05:56 +09:00)


In [9]:
train_data['text'].nunique(), train_data['label'].nunique()

(89329, 2)

time: 93 ms (started: 2021-12-19 20:05:56 +09:00)


In [10]:
train_data.drop_duplicates(subset=['text'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거

time: 62 ms (started: 2021-12-19 20:05:56 +09:00)


C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/2746220271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
print('총 샘플의 수 :',len(train_data))

총 샘플의 수 : 89329
time: 0 ns (started: 2021-12-19 20:05:56 +09:00)


In [12]:
test_data.drop_duplicates(subset=['text'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
print('총 테스트 샘플의 수 :',len(test_data))

총 테스트 샘플의 수 : 9992
time: 15 ms (started: 2021-12-19 20:05:56 +09:00)


C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/224691532.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
train_data['text'] = train_data['text'].str.replace("[^가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
train_data[:5]

C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/1179348747.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/1179348747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,text
15241,0,게임 자체는 개성있고 괜찮지만 매칭이 안 된다 비슷한 등급에 동접자 수가 손가락으로...
95040,0,뭔가 애매하다 금주법시대의 분위기는 좋은데 경영파트는 너무 쉽고 전투파트는 너무 어...
7189,0,그냥 개새끼
91205,0,정말 진짜 하지마세요 답답해서 미치겠는데 죽겠어요 떨어지면
89491,1,중세시대 군주가 되어 플레이하는 시뮬레이션 게임입니다 유로파나 문명을 좋아한다면 이...


time: 141 ms (started: 2021-12-19 20:05:56 +09:00)


In [14]:
test_data.drop_duplicates(subset = ['text'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['text'] = test_data['text'].str.replace("[^가-힣 ]","") # 정규 표현식 수행
test_data['text'] = test_data['text'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['text'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/589794599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/589794599.py:2: FutureWarning: The default value of regex will change from True to False in a future version.


전처리 후 테스트용 샘플의 개수 : 9992
time: 63 ms (started: 2021-12-19 20:05:56 +09:00)


C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/589794599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/589794599.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
C:\Users\nsw20\AppData\Local\Temp/ipykernel_9468/589794599.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\nsw20\Anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy o

### 데이터 토크나이징

In [15]:
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

time: 953 ms (started: 2021-12-19 20:05:56 +09:00)


In [16]:
X_train = []
for sentence in train_data['text']:
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)

time: 4min 29s (started: 2021-12-19 20:05:57 +09:00)


In [17]:
X_test = []
for sentence in test_data['text']:
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

time: 28.7 s (started: 2021-12-19 20:10:26 +09:00)


In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

time: 1.2 s (started: 2021-12-19 20:10:55 +09:00)


In [19]:
print(tokenizer.word_index)

{'게임': 1, '을': 2, '있다': 3, '없다': 4, '되다': 5, '이다': 6, '좋다': 7, '같다': 8, '보다': 9, '로': 10, '재밌다': 11, '안': 12, '겜': 13, '만': 14, '다': 15, '너무': 16, '않다': 17, '아니다': 18, '플레이': 19, '시간': 20, '못': 21, '적': 22, '것': 23, '에서': 24, '때': 25, '안되다': 26, '사다': 27, '추천': 28, '나': 29, '사람': 30, '고': 31, '재미': 32, '거': 33, '해보다': 34, '재미있다': 35, '인': 36, '개': 37, '만들다': 38, '나오다': 39, '생각': 40, '스토리': 41, '그냥': 42, '더': 43, '내': 44, '모르다': 45, '하고': 46, '게': 47, '수': 48, '왜': 49, '많다': 50, '진짜': 51, '갓': 52, '그': 53, '정말': 54, '아': 55, '버그': 56, '가다': 57, '그래픽': 58, '어렵다': 59, '돈': 60, '정도': 61, '요': 62, '그렇다': 63, '하': 64, '함': 65, '들다': 66, '자다': 67, '뭐': 68, '분': 69, '친구': 70, '면': 71, '중': 72, '인데': 73, '해주다': 74, '싶다': 75, '알다': 76, '처음': 77, '환불': 78, '주다': 79, '느낌': 80, '즐기다': 81, '일': 82, '까지': 83, '하나': 84, '좋아하다': 85, '말': 86, '하지만': 87, '멀티': 88, '임': 89, '감': 90, '버리다': 91, '받다': 92, '말다': 93, '오다': 94, '난이도': 95, '구매': 96, '저': 97, '걸': 98, '많이': 99, '실행': 100, '지': 101, '엔딩': 102, '그

In [20]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 32741
등장 빈도가 2번 이하인 희귀 단어의 수: 18788
단어 집합에서 희귀 단어의 비율: 57.38370850004581
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8828232346585076
time: 32 ms (started: 2021-12-19 20:10:56 +09:00)


In [21]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 13954
time: 0 ns (started: 2021-12-19 20:10:57 +09:00)


In [22]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

time: 2.03 s (started: 2021-12-19 20:10:57 +09:00)


In [23]:
print(X_train[:3])

[[1, 130, 1164, 3, 110, 430, 12, 5, 423, 4016, 884, 695, 528, 1369, 3640, 48, 3, 23, 8, 83, 696, 162, 476, 668, 10, 507, 6531, 3, 1705, 5], [216, 1186, 203, 2101, 294, 7, 2016, 574, 2206, 16, 198, 191, 2206, 16, 59, 103, 669, 40, 57, 303], [42, 1218]]
time: 0 ns (started: 2021-12-19 20:10:59 +09:00)


In [24]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

time: 15 ms (started: 2021-12-19 20:10:59 +09:00)


In [25]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

time: 32 ms (started: 2021-12-19 20:10:59 +09:00)


In [26]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

88978
88978
time: 32 ms (started: 2021-12-19 20:10:59 +09:00)


C:\Users\nsw20\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [27]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

time: 0 ns (started: 2021-12-19 20:10:59 +09:00)


In [28]:
max_len = 35
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 35 이하인 샘플의 비율: 95.90011013958507
time: 31 ms (started: 2021-12-19 20:10:59 +09:00)


In [29]:
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

time: 343 ms (started: 2021-12-19 20:10:59 +09:00)


### 학습 진행

In [30]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

time: 0 ns (started: 2021-12-19 20:10:59 +09:00)


In [36]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 100)         1395400   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,512,777
Trainable params: 1,512,777
Non-trainable params: 0
_________________________________________________________________
time: 453 ms (started: 2021-12-19 20:23:43 +09:00)


In [37]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('C:\\Users\\nsw20\\best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

time: 0 ns (started: 2021-12-19 20:23:45 +09:00)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train,
                    y_train,
                    epochs= 20,
                    batch_size=64, # 512, 100, 60 
                    validation_data=(X_test, y_test),
                    callbacks = [es, mc])

Epoch 1/20
1390/1391 [============================>.] - ETA: 0s - loss: 0.4895 - acc: 0.7629
Epoch 00001: val_acc improved from -inf to 0.78533, saving model to C:\Users\nsw20\best_model.h5
1391/1391 [==============================] - 84s 59ms/step - loss: 0.4895 - acc: 0.7629 - val_loss: 0.4510 - val_acc: 0.7853
Epoch 2/20
1390/1391 [============================>.] - ETA: 0s - loss: 0.4125 - acc: 0.8127
Epoch 00002: val_acc improved from 0.78533 to 0.79223, saving model to C:\Users\nsw20\best_model.h5
1391/1391 [==============================] - 83s 60ms/step - loss: 0.4125 - acc: 0.8127 - val_loss: 0.4527 - val_acc: 0.7922
Epoch 3/20
1390/1391 [============================>.] - ETA: 0s - loss: 0.3618 - acc: 0.8392
Epoch 00003: val_acc did not improve from 0.79223
1391/1391 [==============================] - 84s 61ms/step - loss: 0.3618 - acc: 0.8392 - val_loss: 0.4670 - val_acc: 0.7877
Epoch 4/20
1390/1391 [============================>.] - ETA: 0s - loss: 0.3134 - acc: 0.8629
Epoch 